# Uniquely Identifying Particles With Hashes

In many cases, one can just identify particles by their position in the particle array, e.g. using ``sim.particles[5]``. However, in cases where particles might get reordered in the particle array finding a particle might be difficult. This is why we added a *hash* attribute to particles.

In REBOUND particles might get rearranged when a tree code is used for the gravity or collision routine, when particles merge, when a particle leaves the simulation box, or when you manually remove or add particles. In general, therefore, the user should not assume that particles stay at the same index or in the same location in memory.  The reliable way to access particles is to assign them hashes and to access particles through them. 

In this example, we show the basic usage of the *hash* attribute, which is an unsigned integer.

In [1]:
import rebound
sim = rebound.Simulation()
sim.add(m=1., hash=999)
sim.add(a=0.4, hash="mercury")
sim.add(a=1., hash="earth")
sim.add(a=5., hash="jupiter")

We can now not only access the Earth particle with:

In [2]:
sim.particles[2]

<rebound.Particle object, m=0.0 x=1.0 y=0.0 z=0.0 vx=0.0 vy=1.0 vz=0.0>

but also with

In [3]:
sim.particles["earth"]

<rebound.Particle object, m=0.0 x=1.0 y=0.0 z=0.0 vx=0.0 vy=1.0 vz=0.0>

We can remove the ``mercury`` particle with 

In [4]:
del sim.particles[1]

How do we now access the last particle `jupiter`?

In [4]:
sim.particles[-1]

<rebound.Particle object, m=0.0 x=5.0 y=0.0 z=0.0 vx=0.0 vy=0.4472135954999579 vz=0.0>

This gave us back the `earth` particle, so it seems like we lost `jupiter`, but in fact it got swapped into the index vacated by the `mercury` particle we deleted:

In [5]:
sim.particles[1]

<rebound.Particle object, m=0.0 x=0.4 y=0.0 z=0.0 vx=0.0 vy=1.5811388300841898 vz=0.0>

Rather than keep track of how particles get moved around as they get deleted or rearranged under the hood, we can always reliably access particles through their hash:

In [6]:
sim.particles["jupiter"]

<rebound.Particle object, m=0.0 x=5.0 y=0.0 z=0.0 vx=0.0 vy=0.4472135954999579 vz=0.0>

# Details

We can also access particles through their hash directly.  However, to differentiate from passing an integer index, we have to first cast the hash to a `ctypes.c_uint32` (the underlying C datatype):

In [7]:
from ctypes import c_uint32 as u
sim.particles[u(999)]

<rebound.Particle object, m=1.0 x=0.0 y=0.0 z=0.0 vx=0.0 vy=0.0 vz=0.0>

which corresponds to `particles[0]` as it should.  `sim.particles[999]` would try to access index 999, which doesn't exist in the simulation, and REBOUND would raise an AttributeError.

When we above set the hash to a string, REBOUND converted this to an unsigned integer using the `rebound.hash` function:

In [10]:
rebound.hash("earth")

c_uint(1424801690)

In [11]:
sim.particles[2].hash

c_uint(1424801690)

The hash attribute always returns the appropriate unsigned integer ctypes type. (Depending on your computer architecture, `ctypes.c_uint32` can be an alias for another `ctypes` type).

So we could also access the earth with:

In [8]:
sim.particles[u(1424801690)]

<rebound.Particle object, m=0.0 x=1.0 y=0.0 z=0.0 vx=0.0 vy=1.0 vz=0.0>

The numeric hashes could be useful in cases where you have a lot of particles you don't want to assign individual names, but you still need to keep track of them individually as they get rearranged:

In [9]:
for i in range(1,100):
    sim.add(m=0., a=i, hash=i)

In [10]:
sim.particles[99].a

96.0

In [11]:
sim.particles[u(99)].a

98.99999999999999

# Possible Pitfalls

The user is responsible for making sure the hashes are unique.  If two particles share the same hash, you could get either one when you access them using their hash (in most cases the first hit in the `particles` array).  Two random strings used for hashes have a $\sim 10^{-9}$ chance of clashing.

Due to details of the `ctypes` library, comparing two `ctypes.c_uint32` instances for equality fails:

In [12]:
u(32) == u(32)

False

You have to compare the value

In [13]:
u(32).value == u(32).value

True

See the docs for further information: https://docs.python.org/2/library/ctypes.html